In [1]:
exec(open("../../../python/FNC_init.py").read())

[**Demo %s**](#demo-int-extrap)

We estimate $\displaystyle\int_0^2 x^2 e^{-2x}\, dx$ using extrapolation. First we use `quadgk` to get an accurate value.

In [2]:
from scipy.integrate import quad
f = lambda x: x**2 * exp(-2 * x)
a = 0
b = 2
I, errest = quad(f, a, b, epsabs=1e-13, epsrel=1e-13)
print(f"Integral = {I:.14f}")

Integral = 0.19047417361161


We start with the trapezoid formula on $n=N$ nodes.

In [3]:
N = 20    # the coarsest formula
n = N
h = (b - a) / n
t = h * arange(n + 1)
y = f(t)

We can now apply weights to get the estimate $T_f(N)$.

In [4]:
T = zeros(3)
T[0] = h * (sum(y[1:-1]) + y[0] / 2 + y[-1] / 2)
print(f"error (2nd order): {I - T[0]:.2e}")

error (2nd order): 6.27e-05


Now we double to $n=2N$, but we only need to evaluate $f$ at every other interior node and apply {eq}`nc-doubling`.

In [5]:
n = 2 * n
h = h / 2
t = h * arange(n + 1)
T[1] = T[0] / 2 + h * sum(f(t[1:-1:2]))
print("error (2nd order):", I - T[:2])

error (2nd order): [6.27236723e-05 1.53677521e-05]


As expected for a second-order estimate, the error went down by a factor of about 4. We can repeat the same code to double $n$ again.

In [6]:
n = 2 * n
h = h / 2
t = h * arange(n + 1)
T[2] = T[1] / 2 + h * sum(f(t[1:-1:2]))
print("error (2nd order):", I - T[:3])

error (2nd order): [6.27236723e-05 1.53677521e-05 3.82230697e-06]


Let us now do the first level of extrapolation to get results from Simpson's formula. We combine the elements `T[i]` and `T[i+1]` the same way for $i=1$ and $i=2$.

In [7]:
S = array([(4 * T[i + 1] - T[i]) / 3 for i in range(2)])
print("error (4th order):", I - S)

error (4th order): [-4.17554646e-07 -2.61747412e-08]


With the two Simpson values $S_f(N)$ and $S_f(2N)$ in hand, we can do one more level of extrapolation to get a sixth-order accurate result.

In [8]:
R = (16 * S[1] - S[0]) / 15
print("error (6th order):", I - R)

error (6th order): -8.274758656057202e-11


We can make a triangular table of the errors:

In [9]:
err = nan * ones((3, 3))
err[0, :] = I - T
err[1, 1:] = I - S
err[2, 2] = I - R
results = PrettyTable(["2nd order", "4th order", "6th order"])
results.add_rows(err.T)
print(results)

+------------------------+-------------------------+------------------------+
|       2nd order        |        4th order        |       6th order        |
+------------------------+-------------------------+------------------------+
| 6.272367234608223e-05  |           nan           |          nan           |
| 1.5367752102174448e-05 | -4.1755464580406354e-07 |          nan           |
| 3.822306969658573e-06  | -2.6174741207807273e-08 | -8.274758656057202e-11 |
+------------------------+-------------------------+------------------------+


If we consider the computational time to be dominated by evaluations of $f$, then we have obtained a result with about twice as many accurate digits as the best trapezoid result, at virtually no extra cost.